In [164]:
%reset -f
%clear


In [165]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [166]:
# !pip -q install pyomo pandas highspy
# # backup solver (GLPK) in case HiGHS isn’t available for any reason
# !apt-get -qq update
# !apt-get -qq install -y glpk-utils

In [167]:
# Nodes(charging Station candidate sites) data file created
nodes = pd.DataFrame({
    "id": list(range(1, 16)),
    "name": [
        "Beijing North Railway Station",
        "Peking University People's Hospital",
        "Beijing Jiaotong University",
        "Beijing Zoo",
        "Beijing Univ. of Civil Eng. & Architecture",
        "Capital Stadium",
        "Fuchengmen",
        "Xinjiekou",
        "Jishuitan",
        "Navy Hospital",
        "Capital Normal University",
        "South of Baishiqiao",
        "North Campus of Beijing Normal University",
        "Beihai Hospital",
        "Beihai Park"
    ],
    "Cj": [   # basic construction cost
        52000, 57000, 54600, 49000, 52500,
        49000, 45400, 47000, 44000, 56000,
        55000, 49000, 53000, 53000, 49500
    ],
    "PD": [1.0]*15,                # Unit Charging Price
    "lambda_mu_factor": [1.0]*15   # keep =1.0 unless heterogeneous arrivals
})


# Path in Google Drive
file_path = "/content/drive/My Drive/colab_note/location_optimization//data/nodes.csv"

# Save to CSV
nodes.to_csv(file_path, index=False)


In [168]:
# Demand data
demand = pd.DataFrame({
    "i": [1, 2, 3, 4],
    "name": ["Town 1", "Town 2", "Town 3", "Town 4"],
    "mi": [1600, 1500, 2000, 1300],  # total BEVs per town
    "alpha": [1.0]*4,                # perception weight of the charging cost
    "beta": [1.0]*4,                 # perception weight of the access cost
    "QkWh": [0.15]*4                 # power consumption per km distance travelling
})

# Distances with numeric town IDs and station IDs

distances = pd.DataFrame([
    # Town 1 = Yanjiao
    {"i": 1, "j": 1,  "dij_km": 47},
    {"i": 1, "j": 2,  "dij_km": 24},
    {"i": 1, "j": 3,  "dij_km": 49},
    {"i": 1, "j": 4,  "dij_km": 49},
    {"i": 1, "j": 5,  "dij_km": 48},
    {"i": 1, "j": 6,  "dij_km": 48},
    {"i": 1, "j": 7,  "dij_km": 48},
    {"i": 1, "j": 8,  "dij_km": 45},
    {"i": 1, "j": 9,  "dij_km": 44},
    {"i": 1, "j": 10, "dij_km": 52},
    {"i": 1, "j": 11, "dij_km": 59},
    {"i": 1, "j": 12, "dij_km": 55},
    {"i": 1, "j": 13, "dij_km": 55},
    {"i": 1, "j": 14, "dij_km": 47},
    {"i": 1, "j": 15, "dij_km": 55},

    # Town 2 = Lixian (Daxing)
    {"i": 2, "j": 1,  "dij_km": 59},
    {"i": 2, "j": 2,  "dij_km": 55},
    {"i": 2, "j": 3,  "dij_km": 58},
    {"i": 2, "j": 4,  "dij_km": 63},
    {"i": 2, "j": 5,  "dij_km": 55},
    {"i": 2, "j": 6,  "dij_km": 60},
    {"i": 2, "j": 7,  "dij_km": 55},
    {"i": 2, "j": 8,  "dij_km": 57},
    {"i": 2, "j": 9,  "dij_km": 58},
    {"i": 2, "j": 10, "dij_km": 60},
    {"i": 2, "j": 11, "dij_km": 57},
    {"i": 2, "j": 12, "dij_km": 57},
    {"i": 2, "j": 13, "dij_km": 59},
    {"i": 2, "j": 14, "dij_km": 60},
    {"i": 2, "j": 15, "dij_km": 60},

    # Town 3 = Daanshan (Fangshan)
    {"i": 3, "j": 1,  "dij_km": 77},
    {"i": 3, "j": 2,  "dij_km": 77},
    {"i": 3, "j": 3,  "dij_km": 75},
    {"i": 3, "j": 4,  "dij_km": 77},
    {"i": 3, "j": 5,  "dij_km": 75},
    {"i": 3, "j": 6,  "dij_km": 76},
    {"i": 3, "j": 7,  "dij_km": 74},
    {"i": 3, "j": 8,  "dij_km": 73},
    {"i": 3, "j": 9,  "dij_km": 78},
    {"i": 3, "j": 10, "dij_km": 77},
    {"i": 3, "j": 11, "dij_km": 72},
    {"i": 3, "j": 12, "dij_km": 71},
    {"i": 3, "j": 13, "dij_km": 72},
    {"i": 3, "j": 14, "dij_km": 80},
    {"i": 3, "j": 15, "dij_km": 77},

    # Town 4 = Yanshou (Changping)
    {"i": 4, "j": 1,  "dij_km": 47},
    {"i": 4, "j": 2,  "dij_km": 47},
    {"i": 4, "j": 3,  "dij_km": 46},
    {"i": 4, "j": 4,  "dij_km": 50},
    {"i": 4, "j": 5,  "dij_km": 48},
    {"i": 4, "j": 6,  "dij_km": 49},
    {"i": 4, "j": 7,  "dij_km": 48},
    {"i": 4, "j": 8,  "dij_km": 46},
    {"i": 4, "j": 9,  "dij_km": 45},
    {"i": 4, "j": 10, "dij_km": 51},
    {"i": 4, "j": 11, "dij_km": 51},
    {"i": 4, "j": 12, "dij_km": 51},
    {"i": 4, "j": 13, "dij_km": 43},
    {"i": 4, "j": 14, "dij_km": 46},
    {"i": 4, "j": 15, "dij_km": 47},
])

#distances["dij_km"] = distances["dij_km"] / 25.0

# -------------------------------------------------------
# Create charging-station → destination distances (dd)
# -------------------------------------------------------

# Copy the origin station matrix
distances_dd = distances.copy()

# Rename the distance column
distances_dd.rename(columns={"dij_km": "dd_km"}, inplace=True)

# fixed proportion of origin to charging station:
distances_dd["dd_km"] = distances_dd["dd_km"] * 0.9   # e.g. always 90 % of dij

# Save to Drive

demand_path = "/content/drive/My Drive/colab_note/location_optimization/data/demand.csv"
demand.to_csv(demand_path, index=False)

distances_path = "/content/drive/My Drive/colab_note/location_optimization/data/distances.csv"
distances.to_csv(distances_path, index=False)
dd_path = "/content/drive/My Drive/colab_note/location_optimization/data/distances_dd.csv"
distances_dd.to_csv(dd_path, index=False)


In [169]:
%%writefile ev_station_siting.py
import pandas as pd
from pyomo.environ import (
    ConcreteModel, Var, Objective, Constraint, Set, Param, NonNegativeIntegers,
    Binary, minimize, value, SolverFactory, Reals, NonNegativeReals
)

# ----------------------------
# Config () to match the paper)
# ----------------------------
CONFIG = {
    # Planning knobs
    "P": 7,                  # number of stations to open
    "TD_scalar": 75,         # tolerable distance to destination (km)  (paper uses 2.5 km)
    "r": 0.10,               # discount rate
    "T": 10,                 # operating period (years)
    "mu0": 0.20,             # management cost fraction
    "eta": 1.0,              # users’ annualization factor

    # Energy / range parameters for (9), (11), (12)
    "DR_km": 100.0,           # driving range limit origin to charging station (km)
    "e_kwh_per_km": 0.15,   # energy use per km (kWh/km)
    "Q0_kWh": 15,          # initial available energy before charging (kWh)

    # Costs
    "Pc": 10000.0,           # price per charger ($/charger)
    "C_access": 0.40,   # $ per km

    # Data files
    "nodes_csv": "/content/drive/My Drive/colab_note/location_optimization/data/nodes.csv",
    "demand_csv": "/content/drive/My Drive/colab_note/location_optimization/data/demand.csv",
    "dist_csv": "/content/drive/My Drive/colab_note/location_optimization/data/distances.csv",       # origin to station  (dij_km)
    "dist_dd_csv": "/content/drive/My Drive/colab_note/location_optimization/data/distances_dd.csv",# station to destination (dd_km)

    # Solver preference list
    "solvers": ["appsi_highs", "highs", "glpk"],

    # Big-M & bounds
    "Nmax": 20000,          # big M for charger capacity upper bound
    "Nmin_floor": 20         # at least 20 chargers if a station is opened
}

# ----------------------------
# Load data
# ----------------------------
def load_data(cfg):
    nodes = pd.read_csv(cfg["nodes_csv"])
    demand = pd.read_csv(cfg["demand_csv"])
    distances = pd.read_csv(cfg["dist_csv"])          # origin -> station
    distances_dd = pd.read_csv(cfg["dist_dd_csv"])    # station -> destination place
    return nodes, demand, distances, distances_dd

# ----------------------------
# Build model (paper-mimic MILP)
# ----------------------------
def build_model(nodes_df, demand_df, dist_df, dist_dd_df, cfg):
    m = ConcreteModel()

    # Sets
    m.J = Set(initialize=list(nodes_df["id"]))      # candidate stations
    m.I = Set(initialize=list(demand_df["i"]))      # user groups

    # Parameters from data
    id_to_Cj = dict(zip(nodes_df["id"], nodes_df["Cj"]))     # basic construction cost per charger
    id_to_PD = dict(zip(nodes_df["id"], nodes_df["PD"]))     # unit charging price at j
    i_to_mi   = dict(zip(demand_df["i"], demand_df["mi"]))   # vehicles in group i
    i_to_a    = dict(zip(demand_df["i"], demand_df["alpha"]))# perception weight on price
    i_to_b    = dict(zip(demand_df["i"], demand_df["beta"])) # perception weight on distance
    i_to_Q    = dict(zip(demand_df["i"], demand_df["QkWh"])) # power consumption per km distance travelling

    # Distances
    # origin to station
    dij = {(int(r["i"]), int(r["j"])): float(r["dij_km"]) for _, r in dist_df.iterrows()}
    # station to destination (dd_km)
    dd  = {(int(r["i"]), int(r["j"])): float(r["dd_km"])  for _, r in dist_dd_df.iterrows()}

    # Params
    m.dij = Param(m.I, m.J, initialize=lambda mdl,i,j: dij[(i,j)], within=Reals)  # origin to station
    m.dd  = Param(m.I, m.J, initialize=lambda mdl,i,j: dd[(i,j)],  within=Reals)  # station to destination

    m.P = Param(initialize=cfg["P"])
    m.TD = Param(initialize=cfg["TD_scalar"])   # tolerable distance to destination
    m.r = Param(initialize=cfg["r"])
    m.T = Param(initialize=cfg["T"])
    m.mu0 = Param(initialize=cfg["mu0"])
    m.eta = Param(initialize=cfg["eta"])
    m.Pc = Param(initialize=cfg["Pc"])

    # Energy / range params
    m.DR = Param(initialize=cfg["DR_km"])              # driving range (km)
    m.e_km = Param(initialize=cfg["e_kwh_per_km"])     # kWh per km
    m.Q0 = Param(m.I, initialize={i: cfg["Q0_kWh"] for i in m.I})   # initial energy EVs per town

    m.Cj = Param(m.J, initialize=id_to_Cj)
    m.PD = Param(m.J, initialize=id_to_PD) # Unit Charging Price
    m.mi = Param(m.I, initialize=i_to_mi) # no. of BEVs in each town
    m.alpha = Param(m.I, initialize=i_to_a)
    m.beta = Param(m.I, initialize=i_to_b)
    m.Q = Param(m.I, initialize=i_to_Q) # power consumption per km distance travelling
    m.C_access = Param(initialize=cfg["C_access"])

    # Annuity factor
    def annuity_factor():
        r = float(value(m.r))
        T = int(value(m.T))
        num = r * (1.0 + r)**T
        den = (1.0 + r)**T - 1.0
        return num/den if den > 0 else 1.0
    A = annuity_factor()

    # Decision variables
    m.X = Var(m.J, within=Binary)                  # open charging station?
    m.Y = Var(m.I, m.J, within=Binary)             # choose charging station?
    m.N = Var(m.J, within=NonNegativeIntegers)     # number of chargers
    m.Qchg = Var(m.I, m.J, within=NonNegativeReals)# charged energy (kWh)  (constraint (12))

    # Objective function(annualized station cost + user cost)

    def obj_rule(mdl):
        station_annual = A * sum((1.0 + mdl.mu0) * (mdl.Cj[j] + mdl.Pc) * mdl.N[j] for j in mdl.J)
        users_annual = mdl.eta * sum(
            mdl.mi[i] * (mdl.beta[i] * mdl.dd[i,j]* mdl.C_access * mdl.Y[i,j]+ mdl.alpha[i] * mdl.PD[j] * mdl.Q[i])
            for i in mdl.I for j in mdl.J
        )
        return station_annual + users_annual
    m.OBJ = Objective(rule=obj_rule, sense=minimize)

    # --------- Constraints ---------
    # (2) Each user picks exactly one station
    def assign_once_rule(mdl, i):
        return sum(mdl.Y[i,j] for j in mdl.J) == 1
    m.AssignOnce = Constraint(m.I, rule=assign_once_rule)

    # (3) Exactly P stations
    def p_rule(mdl):
        return sum(mdl.X[j] for j in mdl.J) == mdl.P
    m.PStations = Constraint(rule=p_rule)

    # (4) Linking: can choose j station only if j station is open
    def link_rule(mdl, i, j):
        return mdl.Y[i,j] <= mdl.X[j]
    m.Link = Constraint(m.I, m.J, rule=link_rule)

    # (5) If a station is opened, it must have at least 1 charger (redundant with min-20)
    def open_implies_some_chargers(mdl, j):
        return mdl.N[j] >= mdl.X[j]
    m.OpenNeedsCharger = Constraint(m.J, rule=open_implies_some_chargers)

    # Big-M upper bound (forces N=0 when X=0)
    def chargers_max_rule(mdl, j):
        return mdl.N[j] <= cfg["Nmax"] * mdl.X[j]
    m.NmaxLink = Constraint(m.J, rule=chargers_max_rule)

    # (7) ≥ 20 chargers if opened
    def min20_rule(mdl, j):
        return mdl.N[j] >= cfg["Nmin_floor"] * mdl.X[j]
    m.MinTwentyIfOpen = Constraint(m.J, rule=min20_rule)

    # (8) Capacity: chargers ≥ total vehicles assigned
    def capacity_rule(mdl, j):
        return mdl.N[j] >= sum(mdl.mi[i] * mdl.Y[i,j] for i in mdl.I)
    m.CapacityCoversAssigned = Constraint(m.J, rule=capacity_rule)
    # (8b) Maximum number of chargers per station
    def max_chargers_rule(mdl, j):
        return mdl.N[j] <= 2000
    m.MaxChargersLimit = Constraint(m.J, rule=max_chargers_rule)

    # (10) Tolerable distance (use dd: station -> destination)
    def td_rule(mdl, i):
        return sum(mdl.dd[i,j] * mdl.Y[i,j] for j in mdl.J) <= mdl.TD
    m.TolerableDistance = Constraint(m.I, rule=td_rule)

    # (9) Driving-range feasibility (origin -> station)
    # d_oViDj * Y_ij ≤ DR  (linear; forbids choosing stations beyond range)
    def driving_range_rule(mdl, i, j):
        return mdl.dij[i,j] * mdl.Y[i,j] <= mdl.DR
    m.DrivingRange = Constraint(m.I, m.J, rule=driving_range_rule)

    # (11) Energy needed for the (round) trip to/from station, minus initial energy
    # Qchg_ij ≥ 2 * e * d_oViDj * Y_ij - Q0_i
    def energy_need_rule(mdl, i, j):
        return mdl.Qchg[i,j] >= 2.0 * mdl.e_km * mdl.dij[i,j] * mdl.Y[i,j] - mdl.Q0[i]
    m.EnergyNeed = Constraint(m.I, m.J, rule=energy_need_rule)

    # (12) Nonnegativity already enforced by domain: Qchg ∈ NonNegativeReals

    return m

# ----------------------------
# Solve & report
# ----------------------------
def solve_and_report(model, cfg, nodes_df, demand_df, dist_df, dist_dd_df):
    # Pick a solver
    solver = None
    for s in cfg["solvers"]:
        try:
            cand = SolverFactory(s)
            if cand is not None and cand.available():
                solver = cand
                print(f"Using solver: {s}")
                break
        except Exception:
            pass
    if solver is None:
        raise RuntimeError("No suitable MILP solver found. Install HiGHS (highspy) or GLPK.")

    res = solver.solve(model, tee=False)

    # Extract
    X = {j: int(round(value(model.X[j]))) for j in model.J}
    N = {j: int(round(value(model.N[j]))) for j in model.J}
    Y = {(i,j): int(round(value(model.Y[i,j]))) for i in model.I for j in model.J}
    opened = [j for j,x in X.items() if x==1]
    total = value(model.OBJ)

    print("\n=== Optimal Solution (Paper-Mimic with 9,11,12) ===")
    print(f"Total annualized cost: {total:,.2f}")
    print(f"Stations opened (P={value(model.P)}): {opened}")

    print("\nPer-site details:")
    for j in model.J:
        namej = nodes_df.loc[nodes_df["id"]==j, "name"].values[0]
        print(f"  j={j:>2}  {namej:25s}  X={X[j]}  N={N[j]}")

    print("\nAssignments (each i chooses exactly one j):")
    for i in model.I:
        for j in model.J:
            if Y[(i,j)]==1:
                iname = demand_df.loc[demand_df["i"]==i, "name"].values[0]
                jname = nodes_df.loc[nodes_df["id"]==j, "name"].values[0]
                dij_val = float(dist_df[(dist_df["i"]==i) & (dist_df["j"]==j)]["dij_km"].values[0])   # origin -> station
                dd_val  = float(dist_dd_df[(dist_dd_df["i"]==i) & (dist_dd_df["j"]==j)]["dd_km"].values[0]) # station -> destination
                print(f"  i={i:>2} {iname:18s} -> j={j:>2} {jname:25s}   do={dij_val:.2f} km, dd={dd_val:.2f} km")

# ----------------------------
# Main
# ----------------------------
if __name__ == "__main__":
    nodes, demand, distances, distances_dd = load_data(CONFIG)
    m = build_model(nodes, demand, distances, distances_dd, CONFIG)
    solve_and_report(m, CONFIG, nodes, demand, distances, distances_dd)


Overwriting ev_station_siting.py


In [170]:
from pathlib import Path

p = Path("ev_station_siting.py")
code = p.read_text()

# 1) Add Reals to the imports from pyomo.environ
code = code.replace(
    "Binary, minimize, value, SolverFactory",
    "Binary, minimize, value, SolverFactory, Reals"
)

# 2) Replace the incorrect 'within=float' with the proper Pyomo domain
code = code.replace("within=float", "within=Reals")

p.write_text(code)
print("Patched ev_station_siting.py")


Patched ev_station_siting.py


In [171]:
!python ev_station_siting.py

Using solver: appsi_highs

=== Optimal Solution (Paper-Mimic with 9,11,12) ===
Total annualized cost: 70,924,684.38
Stations opened (P=7): [4, 6, 7, 8, 9, 12, 15]

Per-site details:
  j= 1  Beijing North Railway Station  X=0  N=0
  j= 2  Peking University People's Hospital  X=0  N=0
  j= 3  Beijing Jiaotong University  X=0  N=0
  j= 4  Beijing Zoo                X=1  N=20
  j= 5  Beijing Univ. of Civil Eng. & Architecture  X=0  N=0
  j= 6  Capital Stadium            X=1  N=1300
  j= 7  Fuchengmen                 X=1  N=1600
  j= 8  Xinjiekou                  X=1  N=1500
  j= 9  Jishuitan                  X=1  N=2000
  j=10  Navy Hospital              X=0  N=0
  j=11  Capital Normal University  X=0  N=0
  j=12  South of Baishiqiao        X=1  N=20
  j=13  North Campus of Beijing Normal University  X=0  N=0
  j=14  Beihai Hospital            X=0  N=0
  j=15  Beihai Park                X=1  N=20

Assignments (each i chooses exactly one j):
  i= 1 Town 1             -> j= 7 Fuchengmen     